In [2]:
import os
import yaml
import json
import torch
import shutil
import numpy as np
import pandas as pd
import ultralytics
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

print("Torch CUDA is available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

Torch CUDA is available: True
GPU name: NVIDIA GeForce RTX 3090


In [3]:
input_dir = 'annotated_outdoor_yolo'
output_dir = 'dataset_yolo'

os.makedirs(os.path.join(output_dir), exist_ok=True)

images_dir = os.path.join(input_dir, 'images')
labels_dir = os.path.join(input_dir, 'labels')

## Setup

In [4]:
# Get the list of image files (assuming all formats are .jpg, change if needed)
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".jpg")])

# Create dataset indices
dataset_size = len(image_files)
indices = list(range(dataset_size))

# Split into train (70%), val (10%), test (20%)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=199742069)
train_indices, val_indices = train_test_split(train_indices, test_size=0.125, random_state=199742069)

# Store indices in a dictionary
split_data = {
    "train": train_indices,
    "val": val_indices,
    "test": test_indices
}

print(f"Train set size: {len(train_indices)} - {len(train_indices)/len(indices)*100:.2f}%")
print(f"Validation set size: {len(val_indices)} - {len(val_indices)/len(indices)*100:.2f}%")
print(f"Test set size: {len(test_indices)} - {len(test_indices)/len(indices)*100:.2f}%\n")

# Save the split indices as JSON
json_split_path = os.path.join(output_dir, "dataset_split.json")
with open(json_split_path, "w") as f:
    json.dump(split_data, f, indent=4)

print(f"Dataset split saved to {json_split_path}")

Train set size: 217 - 69.77%
Validation set size: 31 - 9.97%
Test set size: 63 - 20.26%

Dataset split saved to dataset_yolo/dataset_split.json


In [5]:
# List and sort both image and label files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

# Sanity check
assert len(image_files) == len(label_files), "Image and label counts do not match!"

# Define dataset splits
splits = ["train", "val", "test"]

# Create output directories
for split in splits:
    for category in ["images", "labels"]:
        os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# Loop over your splits
for split, indices in split_data.items():
    for idx in indices:
        image_name = image_files[idx]
        label_name = label_files[idx]

        image_src = os.path.join(images_dir, image_name)
        label_src = os.path.join(labels_dir, label_name)

        image_dest = os.path.join(output_dir, split, "images", image_name)
        label_dest = os.path.join(output_dir, split, "labels", label_name)

        shutil.copy(image_src, image_dest)
        shutil.copy(label_src, label_dest)

# Print final counts for verification
print("\nCopy complete! Summary:\n")
for split in split_data.keys():
    image_folder = os.path.join(output_dir, split, "images")
    label_folder = os.path.join(output_dir, split, "labels")

    num_images = len([f for f in os.listdir(image_folder) if f.endswith('.jpg')]) if os.path.exists(image_folder) else 0
    num_labels = len([f for f in os.listdir(label_folder) if f.endswith('.txt')]) if os.path.exists(label_folder) else 0

    print(f"Split '{split}': {num_images} images, {num_labels} labels.")


Copy complete! Summary:

Split 'train': 217 images, 217 labels.
Split 'val': 31 images, 31 labels.
Split 'test': 63 images, 63 labels.


In [6]:
# Create a .yaml file for the YOLO trainer configuration
dataset_yaml_path = os.path.join(output_dir, 'dataset.yaml')
classes_path = os.path.join(input_dir, 'classes.txt')

with open(classes_path, 'r') as f:
    names = [line.strip() for line in f if line.strip()]

print(names)

# Construct .yaml file to include dataset paths
data_yaml = {
    "names": names,
    "nc": len(names),
    "train": os.path.abspath(os.path.join(output_dir, "train", "images")),
    "val": os.path.abspath(os.path.join(output_dir, "val", "images")),
    "test": os.path.abspath(os.path.join(output_dir, "test", "images"))
}

# Save to .yaml file
with open(dataset_yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"YAML file saved at {dataset_yaml_path}")


['bus', 'bus_station', 'car', 'crossroad', 'person', 'pothole', 'stairs_down', 'stairs_up']
YAML file saved at dataset_yolo/dataset.yaml


## Model Training

In [29]:
model = YOLO('yolov8n.pt')

In [ ]:
# Define parameters for training the YOLO model
yolo_params = {
    'image_size' : 640,
    'batch_size' : 16,
    'epochs' : 200
}

dataset_yaml_path = os.path.join(output_dir, 'dataset.yaml')

# Train the model
results = model.train(
            data=dataset_yaml_path,
            imgsz=yolo_params['image_size'],
            epochs=yolo_params['epochs'],
            batch=yolo_params['batch_size'],
            name=f'yolov8n_outdoor_train',
            project=os.path.join(output_dir, 'runs'),
            device=0,
            patience=0 
        )

New https://pypi.org/project/ultralytics/8.3.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_yolo/dataset.yaml, epochs=100, time=None, patience=0, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=dataset_yolo/runs, name=yolov8n_outdoor_train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, em

train: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/labels.cache... 217 images, 0 backgrounds, 0 corrupt: 100%|██████████| 217/217 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/0274d479-urban_scenes_114.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/02e71862-urban_scenes_72.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/04945f74-urban_scenes_115.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/0839f0e6-urban_scenes_139.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/0ad01010-urban_scenes_106.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/train/images/140c253b-urban_scenes_133.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/user_


val: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/1552f1ae-urban_scenes_58.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/15e532ad-urban_scenes_25.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/16b51f6d-urban_scenes_5.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/22ae1e24-urban_scenes_100.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/4b237d1d-urban_scenes_61.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/val/images/55dfe649-urban_scenes_102.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_M

...

    100/100      1.53G     0.6574     0.4457     0.8836         20        640: 100%|██████████| 28/28 [00:01<00:00, 17.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 25.99it/s]

                   all         31        191      0.866      0.525      0.577      0.358



100 epochs completed in 0.108 hours.
Optimizer stripped from dataset_yolo/runs/yolov8n_outdoor_train5/weights/last.pt, 6.2MB
Optimizer stripped from dataset_yolo/runs/yolov8n_outdoor_train5/weights/best.pt, 6.2MB

Validating dataset_yolo/runs/yolov8n_outdoor_train5/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 20.76it/s]


                   all         31        191      0.757      0.554      0.605      0.389
                   bus          4          9      0.916      0.889      0.956      0.726
           bus_station          6          8      0.861      0.625      0.644      0.475
                   car         18        100      0.761      0.413      0.516      0.309
             crossroad          6         14      0.627      0.286      0.259      0.174
                person         10         25      0.817        0.4      0.519      0.214
               pothole         10         22      0.647      0.318      0.336      0.209
           stairs_down          4          6      0.771      0.667      0.702      0.509
             stairs_up          6          7      0.659      0.831      0.911      0.498
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to dataset_yolo/runs/yolov8n_outdoor_train5


## Test

In [26]:
best_model_path = os.path.join(output_dir,'runs','yolov8n_outdoor_train','weights','best.pt')
best_model = YOLO(best_model_path)

In [27]:
test_results = best_model.val(data=dataset_yaml_path, split='test', name=f'test')
print(f"mAP@50 test results: {test_results.box.map50}")

Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients


val: Scanning /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/labels.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/0b6e2409-urban_scenes_135.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/0d4b1513-urban_scenes_30.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/1d2517df-urban_scenes_67.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/30bfc043-urban_scenes_50.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/3261961d-urban_scenes_32.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/test/images/3ac41843-urban_scenes_51.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /home/user_138/Blind_Guidance_Syste


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.86it/s]


                   all         63        402      0.659      0.473      0.519      0.287
                   bus          8         27       0.58      0.556      0.703      0.428
           bus_station         23         28      0.727      0.763      0.794      0.582
                   car         36        174      0.674      0.466      0.527      0.287
             crossroad         12         23       0.74      0.391       0.46      0.172
                person         17         49      0.633       0.51      0.515      0.238
               pothole         16         63      0.485      0.206      0.209     0.0963
           stairs_down         13         25      0.726       0.52      0.555      0.207
             stairs_up          9         13      0.709      0.376      0.387      0.283
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/test6
mAP@50 test results: 0.5187592425111186


## Export as ONNX

In [28]:
# Export trained model to .onnx to later convert to .blob using BlobConverter
best_model.export(format='onnx', dynamic=True, simplify=True)

Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CPU (AMD Ryzen 9 5900X 12-Core Processor)

PyTorch: starting from 'dataset_yolo/runs/yolov8n_outdoor_train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 9.0s, saved as 'dataset_yolo/runs/yolov8n_outdoor_train/weights/best.onnx' (11.6 MB)

Export complete (9.1s)
Results saved to /home/user_138/Blind_Guidance_System/02_Model_Training/dataset_yolo/runs/yolov8n_outdoor_train/weights
Predict:         yolo predict task=detect model=dataset_yolo/runs/yolov8n_outdoor_train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=dataset_yolo/runs/yolov8n_outdoor_train/weights/best.onnx imgsz=640 data=dataset_yolo/dataset.yaml  
Visualize:       https://netron.app


'dataset_yolo/runs/yolov8n_outdoor_train/weights/best.onnx'